Quick, Draw! 데이터 집합을 이용한 손그림 인식 모델 생성

200219 연구실에서 개인 과제할 때는 1.6버전에서 돌아가던 것을 2.2의 keras model에 넣어서 돌려보려 한다.

데이터셋은 numpy형식으로 된 것을 찾아서 읽어서 사용해볼 생각이다.

By Adrain

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import argparse
import ast
import functools
import sys
import tensorflow as tf
from tensorflow import keras
import numpy as np

###download
Quick Draw에서 제공하는 Google Storage에서 gsutil -m cp 명령어를 통해 로컬에 복사하여 사용한다. 36GB 크기라서 엄청 크다;;

In [ ]:
!mkdir rnn_tutorial_data
!cd rnn_tutorial_data
!gsutil -m cp "gs://quickdraw_dataset/full/numpy_bitmap/*.npy" /content/rnn_tutorial_data
!mkdir train_data
!mkdir test_data
#!mv *.npy ./rnn_tutorial_data

Copying gs://quickdraw_dataset/sketchrnn/The Eiffel Tower.full.npz...
Copying gs://quickdraw_dataset/sketchrnn/The Great Wall of China.full.npz...
Copying gs://quickdraw_dataset/sketchrnn/The Mona Lisa.full.npz...
Copying gs://quickdraw_dataset/sketchrnn/aircraft carrier.full.npz...
Copying gs://quickdraw_dataset/sketchrnn/airplane.full.npz...
Copying gs://quickdraw_dataset/sketchrnn/alarm clock.full.npz...
Copying gs://quickdraw_dataset/sketchrnn/ambulance.full.npz...
Copying gs://quickdraw_dataset/sketchrnn/angel.full.npz...
Copying gs://quickdraw_dataset/sketchrnn/animal migration.full.npz...
Copying gs://quickdraw_dataset/sketchrnn/ant.full.npz...
Copying gs://quickdraw_dataset/sketchrnn/anvil.full.npz...
Copying gs://quickdraw_dataset/sketchrnn/apple.full.npz...
Copying gs://quickdraw_dataset/sketchrnn/arm.full.npz...
Copying gs://quickdraw_dataset/sketchrnn/asparagus.full.npz...
Copying gs://quickdraw_dataset/sketchrnn/axe.full.npz...
Copying gs://quickdraw_dataset/sketchrnn/back

이제 numpy array로 읽어들여야한다. 파일이 너무 많다...

numpy.load로 모든파일을 읽으려 시도하니 RAM 부족으로 터져버렸다...'

12GB 램이 터져버린..

데이터가 (-1,784) -> 28x28 => 즉 아래의 149736, 784 -> 28x28의 그림이 149736개 존재.<br>
그리고 총 데이터는 345개 존재 -> label -> 345개 존재.<br>



각 그림별로 10만개 이상의 그림이 존재한다 -> 이를 전체 학습에 쓰기는 힘들다 <br>=> 램이 너무 작음!
각 그림별로 1만개로 해도 345개면 345만개;;; 램이 버텨주는지 테스트

In [ ]:
import glob
path = './rnn_tutorial_data/*.npy'
temp_array = []
train_array = []
test_array = []
test_label_array = []
cnt = 0
label = 0
train_label_array = []
for filename in glob.glob(path):
  print(filename)
  temp = np.load(filename)
  #temp = temp / temp.max()
  for line in temp:
    if cnt < 50000:
      for i in range(28):
        temp_array.append(line[i*28:(i+1)*28])
        train_label_array.append(label)
      train_array.append(temp_array)
      temp_array = []
      cnt = cnt + 1
    elif cnt < 100000:
      cnt = cnt + 1
    elif cnt < 110000:
      for i in range(28):
        temp_array.append(line[i*28:(i+1)*28])
        test_label_array.append(label)
      test_array.append(temp_array)
      temp_array = []
      cnt = cnt + 1
    else:
      break
  cnt = 0
  label = label + 1
  train_array = np.array(train_array)
  train_label_array = np.array(train_label_array)
  test_array = np.array(test_array)
  test_label_array = np.array(test_label_array)
  print(train_array)
  print(test_array)
  '''
  fname = '/content/train_data'+ filename[19:-4] + '_train'
  np.savez(fname,train_array)
  fname = '/content/train_data' + filename[19:-4] + '_train_label'
  np.savez(fname,train_label_array)
  fname = '/content/test_data' + filename[19:-4] + '_test'
  np.savez(fname,test_array)
  fname = '/content/test_data' + filename[19:-4] + '_test_label'
  np.savez(fname,test_label_array)
  '''
  temp = []
  train_array = []
  test_array = []
  train_label_array = []
  test_label_array = []

'''
train_array = train_array.reshape((-1,28,28,1))
print(train_array.shape)

test_array = test_array.reshape((-1,28,28,1))
print(test_array.shape)
'''
#MODEL 호출

./rnn_tutorial_data/parrot.full.npz
[['test' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''
  '' '' '' '' '' '']
 ['train' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''
  '' '' '' '' '' '']
 ['valid' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''
  '' '' '' '' '' '']]
[]
./rnn_tutorial_data/hot air balloon.full.npz
[['test' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''
  '' '' '' '' '' '']
 ['train' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''
  '' '' '' '' '' '']
 ['valid' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''
  '' '' '' '' '' '']]
[]
./rnn_tutorial_data/angel.full.npz
[['test' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''
  '' '' '' '' '' '']
 ['train' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''
  '' '' '' '' '' '']
 ['valid' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''
  '' '' '' '' '' '']]
[]
./rnn_tutorial_data/baseball.

'\ntrain_array = train_array.reshape((-1,28,28,1))\nprint(train_array.shape)\n\ntest_array = test_array.reshape((-1,28,28,1))\nprint(test_array.shape)\n'

In [ ]:
# Build the tf.keras.Sequential model by stacking layers.
# 선형 layer 모델 -> 층별로 layer를 쌓는다고 생각하면 됨!
def Dnn_Model(x_train,y_train):
  dnn_model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(28, 28,1)),
    keras.layers.Dense(256, activation='relu'),#2D Layer
    keras.layers.Dropout(0.1),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(64, activation='softmax')
  ])
  dnn_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  #dnn_model.fit(x_train,y_train,epoch=10,batch_size=32)

In [ ]:
def Cnn_Model(x_train,y_train):
  cnn_model = keras.Sequential(
    keras.layers.Conv2D(16,3,activation='relu',input_shape(28,28,1)),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Conv2D(32,3,activation='relu'),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Dropout(0.1),
    keras.layers.Conv2D(64,3,activation='relu'),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Dropout(0.1),
    keras.layers.Conv2D(128,3,activation='relu'),
    keras.layers.Flatten(),
    keras.layers.Dense(128,activation='relu'),
    keras.layers.Dense(64,activation='relu'),
    keras.layers.Dense(128,activation='relu'),
    keras.layers.Dense(64,activation='softmax')
  )
  cnn_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  #cnn_model.fit(x_train,y_train,epoch=10,batch_size=32)

###a


어쩔수 없이 tfrecord파일을 읽어서 실행해보자...

In [ ]:
path_tr_data = '/content/drive/My Drive/quickdraw_tutorial_dataset_v1/training.tfrecord-?????-of-?????'
path_ev_data = '/content/drive/My Drive/quickdraw_tutorial_dataset_v1/eval.tfrecord-?????-of-?????'
path_classes = '/content/drive/My Drive/quickdraw_tutorial_dataset_v1/training.tfrecord.classes'

In [ ]:

def get_input_fn(mode, tfrecord_pattern, batch_size):
  """Creates an input_fn that stores all the data in memory.
  Args:
   mode: one of tf.contrib.learn.ModeKeys.{TRAIN, INFER, EVAL}
   tfrecord_pattern: path to a TF record file created using create_dataset.py.
   batch_size: the batch size to output.
  Returns:
    A valid input_fn for the model estimator.
  """

  def _parse_tfexample_fn(example_proto, mode):
    """Parse a single record which is expected to be a tensorflow.Example."""
    feature_to_type = {
        "ink": tf.io.VarLenFeature(dtype=tf.float32),
        "shape": tf.io.FixedLenFeature([2], dtype=tf.int64)
    }
    if mode != tf.estimator.ModeKeys.PREDICT:
      # The labels won't be available at inference time, so don't add them
      # to the list of feature_columns to be read.
      feature_to_type["class_index"] = tf.io.FixedLenFeature([1], dtype=tf.int64)

    parsed_features = tf.io.parse_single_example(example_proto, feature_to_type)
    labels = None
    if mode != tf.estimator.ModeKeys.PREDICT:
      labels = parsed_features["class_index"]
    parsed_features["ink"] = tf.sparse.to_dense(parsed_features["ink"])
    return parsed_features, labels

  def _input_fn():
    """Estimator `input_fn`.
    Returns:
      A tuple of:
      - Dictionary of string feature name to `Tensor`.
      - `Tensor` of target labels.
    """
    dataset = tf.data.TFRecordDataset.list_files(tfrecord_pattern)
    if mode == tf.estimator.ModeKeys.TRAIN:
      dataset = dataset.shuffle(buffer_size=10)
    dataset = dataset.repeat()
    # Preprocesses 10 files concurrently and interleaves records from each file.
    dataset = dataset.interleave(
        tf.data.TFRecordDataset,
        cycle_length=10,
        block_length=1)
    dataset = dataset.map(
        functools.partial(_parse_tfexample_fn, mode=mode),
        num_parallel_calls=10)
    dataset = dataset.prefetch(10000)
    if mode == tf.estimator.ModeKeys.TRAIN:
      dataset = dataset.shuffle(buffer_size=1000000)
    # Our inputs are variable length, so pad them.
    dataset = dataset.padded_batch(
        batch_size, padded_shapes=dataset.output_shapes)
    features, labels = dataset.make_one_shot_iterator().get_next()
    return features, labels

  return _input_fn

In [ ]:
for filename in glob.glob(path_tr_data):
  dataset = tf.data.TFRecordDataset(filename)
for filename in glob.glob(path_ev_data):
  test_dataset = tf.data.TFRecordDataset(filename)
for filename in glob.glob(path_classes):
  label = tf.data.TFRecordDataset(filename)

print(dataset)
print()
print(test_dataset)
print()
print(label)

eagertensor > type을 자동으로 맞춰주는 tensor <br> we can use immediately change to numpy by .numpy()

In [ ]:
feature_to_type = {                      
        "ink": tf.io.VarLenFeature(dtype=tf.float32),                   
        "shape": tf.io.FixedLenFeature([2], dtype=tf.int64)
}

def _parse_function(example_proto):
  # Parse the input `tf.Example` proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, feature_to_type)

parsed_dataset = dataset.map(_parse_function)
len = 0
x_train = []
y_train = []
for element in parsed_dataset
  #print(element.get('ink').values.numpy().shape)
  #print(element.get('shape').numpy().shape)
#print(len)

(344738,)


In [ ]:
#total length : 344738
#maximum label = 1274
model = keras.Sequential(
    #keras.layers.Flatten(),
    keras.layers.Dense(128,activation='relu'),
    keras.layers.Dense(64),
    keras.layers.Dense(128),
    keras.layers.Dense(16),
    keras.layers.Dense(8, activation='softmax')
)
model.summary()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
for element in parsed_dataset:
  model.fit(element.get('ink'),element.get('shape'),epochs=10,batch_size=32)

In [ ]:
np_array = np.array(np_array)
print(len(np_array))
print(np_array)